# Задание 3.1 - Сверточные нейронные сети (Convolutional Neural Networks)

Это последнее задание на numpy, вы до него дожили! Остался последний марш-бросок, дальше только PyTorch.

В этом задании вы реализуете свою собственную сверточную нейронную сеть.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# %env CUPY_ACCELERATORS=cutensor

In [3]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer, ConvolutionalLayer, MaxPoolingLayer, Flattener
from model import ConvNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD, Adam
from metrics import multiclass_accuracy
import cupy as cp

/home/michael/jup_env/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/michael/jup_env/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [4]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train[0].shape

(28, 28)

# Загружаем данные

На этот раз мы не будем их преобразовывать в один вектор, а оставим размерности (num_samples, 32, 32, 3).

In [6]:
from sklearn.model_selection import train_test_split
def prepare_for_neural_network(train_X, test_X):
    train_X = train_X.astype(np.float) / 255.0
    test_X = test_X.astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_X, axis = 0)
    train_X -= mean_image
    test_X -= mean_image
    return train_X, test_X
    
train_X, test_X = prepare_for_neural_network(x_train, x_test)
# Split train into train and val
train_X, val_X, train_y, val_y = train_test_split(train_X, y_train, random_state=42, test_size=0.3)
test_y = y_test

<ipython-input-6-b52e92a3a2f9>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_X = train_X.astype(np.float) / 255.0
<ipython-input-6-b52e92a3a2f9>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_X = test_X.astype(np.float) / 255.0


In [7]:
train_X = train_X.reshape((-1, 28, 28, 1))
val_X = val_X.reshape((-1, 28, 28, 1))
test_X = test_X.reshape((-1, 28, 28, 1))

In [8]:
train_X.shape, val_X.shape, test_X.shape

((42000, 28, 28, 1), (18000, 28, 28, 1), (10000, 28, 28, 1))

## Итак, оверфитим маленький набор данных


In [9]:
# train_X_resized = train_X[:].repeat(8, axis=1).repeat(8, axis=2)
# val_X_resized = val_X[:].repeat(8, axis=1).repeat(8, axis=2)
# train_X_resized.shape

In [10]:
# train_X_resized_gpu = cp.asarray(train_X_resized)
# val_X_resized_gpu = cp.asarray(val_X_resized)
# train_y_gpu = cp.asarray(train_y)
# val_y_gpu = cp.asarray(train_y)

In [11]:
# train_X_resized = train_X[:128].repeat(3, axis=1).repeat(3, axis=2)
# val_X_resized = val_X[:128].repeat(3, axis=1).repeat(3, axis=2)
# train_X_resized.shape

In [12]:
batch_size = 64
train_size = (4200 // batch_size)*batch_size
val_size = (1800 // batch_size)*batch_size

model = ConvNet(input_shape=(224,224,1), n_output_classes=10, conv1_channels=40, conv2_channels=40)
dataset = Dataset(train_X[:train_size], train_y[:train_size], val_X[:val_size], val_y[:val_size])
# dataset = Dataset(train_X[:128], train_y[:128], val_X[:128], val_y[:128])

# model.load_params('best_model')
trainer = Trainer(model, dataset, Adam(), learning_rate=1e-3, num_epochs=50, batch_size=batch_size)

loss_history, train_history, val_history = trainer.fit()
# 656 batches

NameError: name 'grads_X' is not defined

In [ ]:
# del model

In [ ]:
plt.plot(train_history)
plt.plot(val_history)

In [ ]:
plt.plot(loss_history)

In [ ]:
best_model = ConvNet(input_shape=(224,224,1), n_output_classes=10, conv1_channels=40, conv2_channels=40)
best_model.load_params('best_model')

In [ ]:
batch_sizloss_history128
test_size = (10000 // batch_size)*batch_size
# test_size = 128

best_model.predict_accuracy(test_X[:test_size], test_y[:test_size], batch_size)